In [1]:
import sys
import os
import importlib

repo_root = os.path.dirname(os.getcwd())
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)


In [5]:
from labels import *
build_labels(
    os.path.join(repo_root, "data_work/rcs_keys.parquet"),
    os.path.join(repo_root, "data_work/txns_keys.parquet"),
    os.path.join(repo_root, "data_work/rcs_labels.parquet")
)

In [ ]:
# from io_load import *
# rc_labels = load_df_from_pq(os.path.join(repo_root, "data_work/rcs_labels.parquet"))

pq loaded from c:\Users\20878\25Fall\meituan-coupon-roi\data_work/rcs_labels.parquet, shape=(12693100, 26), size=253.15 MB


In [2]:
from importlib import reload
import diagnostics
reload(diagnostics)
from diagnostics import *

diag_df = diag_on_receipts(
    os.path.join(repo_root, "data_work/rcs_labels.parquet"),
    [1000, 10000],  # in units of cents 
    [1000, 10000], # in units of cents
    [10]
)

In [3]:
diag_df

,Price_limit_upper_bin,Coupon_limit_upper_bin,Expiry_span_upper_bin,validity_pc,redeem_fh_pc,redeem_st_pc,receipts_count
0,10000,1000,10 days,98.53,3.69,3.69,7506042
1,1000,1000,10 days,93.70,4.70,4.70,1282715
2,800000,160000,97 days,100.00,1.09,1.09,642
3,899900,189000,10 days,99.99,0.78,0.78,126197
4,300000,10000,10 days,99.23,0.63,0.63,495044
5,999900,1000,90 days,99.22,1.00,0.79,18811
6,300000,10000,120 days,98.57,3.05,2.85,66267
7,0,999900,255 days,100.00,19.20,19.20,125
8,1000,1000,180 days,98.96,28.21,27.18,736701
9,<NA>,<NA>,NaT,NaN,NaN,NaN,0


## Observations
- **High validity rate:** the pc of coupons used/un-used in a valid way within each segment is generally high (>95%). However, the validity rate in several segments are still concerning. The segment with medium Price limit (btwn $10 to $100), medium Coupon amount (btwn $10 to $100), and short expiry span (<10d) contains ~1.7M coupons, but the validity rate is only ~90% (roughly 200 thousand coupons might get abused, indicating ~20000 RNB yuan of potential margin may be lost). So, validity check/pre-screening is quite important when building up the data-driven ROI model.

- **Low redemption rate:** when it comes to the same-user valid redemption, the performances across segments are not good generally, most only have 1%~7%; while several segments stand out: the segment with low Price limit, medium coupon amount, and long expiry span achieves a full horizon redemption rate of ~50%, which is normal, because this is a type of fat meat. Of course, it is also within the expectation to see other "free-lunch-type" coupons with low or almost no price threshold, medium-to-high coupon amount, and long expiry span strikes a relatively high redemption rate. But what is not understandable is that users are generally not able to use coupons with shorter expiry span even when the coupon price threshold is medium-to-low and the coupon amount is medium-to-high, indicating that short-term coupons are not as attractive to their longer-lasting peers. Large-volume segments with low redemption rate are those coupons need to be re-designed, or carefully targetted at specific users in the future.

From the perspective of model training, "High validity, high redemption, high volume" segments are the **high-integrity cohorts**. These are the most reliable for model learning and the most profitable coupon designs.

